## 7.2 모델 구현하기

MNIST 데이터를 CNN으로 학습시키는 모델!


In [1]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot = True)

C:\Users\soual\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

이전까지의 모델들은 입력값을 28X28 = 784로 차원 하나로 구성해왔음 but CNN에서는 2차원 평면으로 구성! (28X28 그대로 구현)

X의 첫 번째 차원 : None = 입력 데이터의 갯수

X의 마지막 차원 : 1 = 특징의 갯수! -> 회색조 이미지!(색상 1개) 

출력값인 10개 분류 & dropout을 위한 placeholder 정의

In [3]:
W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev = 0.01))
L1 = tf.nn.conv2d(X, W1, strides = [1,1,1,1], padding = 'SAME')
L1 = tf.nn.relu(L1)

#### 첫 번째 CNN 계층 구성

1. 3X3 크기의 filter를 가진 convolutional 계층을 만듦

2. filter에 사용할 weight 와 tensorflow가 제공하는 tf.nn.conv2d 함수 사용

위의 코드는 입력층 X와 첫 번째 계층의 weight W1을 갖고 (첫 번째 줄)

오른쪽과 아래쪽으로 한 칸씩 움직이는 32개의 filter를 가진 convolution 계층을 만듦 (두 번째 줄)

이때, padding = 'SAME'은 filter sliding 시 이미지의 외각에서 한 칸 밖으로 움직이는 옵션! -> 이미지의 테두리까지 더 정확하게 평가 가능 (padding을 사용하여 크기가 그대로!)

첫번째 layer의 size가 nXn -> 두번째 layer의 size = n-f+1 <- without pooling

첫번째 layer의 size가 nXn -> 두번째 layer의 size = n+2p-f+1 <- with pooling

same pooling을 하게 되면 p = (f-1)/2

In [4]:
L1 = tf.nn.max_pool(L1, ksize = [1,2,2,1], strides = [1,2,2,1], padding = "SAME")

다음은 pooling 계층을 만듦

앞서 만든 convolution 계층을 입력층으로 사용 -> filter 크기를 2X2로 하는 pooling 계층을 만듦 

strides = [1,2,2,1] : sliding 시 두 칸 씩 움직인다는 옵션! 

convolution layer의 size가 nH X nW X nC -> pooling layer의 size = ((nH-f)/s + 1) X ((nW-f)/s + 1) X nC 

In [5]:
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev = 0.01))
L2 = tf.nn.conv2d(L1, W2, strides = [1,1,1,1], padding = "SAME")
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize = [1,2,2,1], strides = [1,2,2,1], padding = "SAME")

#### 두 번째 계층 구성

W2의 변수 구성 : [3,3,32,64] 

-> 32 : 앞 층의 filter 갯수! => 출력층의 갯수 / 첫 번째 convolution 계층이 찾아낸 이미지의 특징 갯수

In [6]:
W3 = tf.Variable(tf.random_normal([7*7*64, 256], stddev = 0.01))
L3 = tf.reshape(L2, [-1, 7*7*64])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)
L3 = tf.nn.dropout(L3, keep_prob)

#### 세 번째 계층 구성

추출한 feature들을 이용해 10개의 분류를 만들어 내는 계층 구성!

10개의 분류는 1차원 배열 -> 차원을 줄이는 단계를 거쳐야 함! (직전의 pooling 계층 크기 : 7X7X64 -> reshape => 7X7X64크기의 1차원 계층으로 만듦 

##### L3 = tf.reshape(L2, [-1, 7*7*64])

L2의 feature들을 7X7X64의 1차원 배열로 만들어줌 -> L3 = (1,7X7X64)

##### L3 = tf.matmul(L3, W3)

배열 전체를 최종 출력값의 중간 단계인 256개의 뉴런으로 연결하는 신경망 만듦

L3 X W3 = (1,7X7X64) X (7X7X64,256) = (1,256)

##### L3 = tf.nn.relu(L3)

relu! 

-> **fully connected layer** : 인접한 계층의 모든 뉴런과 상호 연결된 계층

##### L3 = tf.nn.dropout(L3, keep_prob)

overfitting 막기 위해 dropout 사용!

In [7]:
W4 = tf.Variable(tf.random_normal([256,10], stddev = 0.01))
model = tf.matmul(L3,W4)

#### 네 번째 계층 구성

L3의 출력값 256개를 받아 최종 출력값인 0~9 label을 갖는 10개의 출력값을 만든다!

In [9]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = model, labels = Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

#optimizer = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)

cost function과 AdamOptimizer를 사용한 최적화 함수를 만듦

In [ ]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(15):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1,28,28,1) # model에 입력값을 전달하기 위해 데이터를 28X28 형태로 재구성해야함!
        
        _, cost_val = sess.run([optimizer, cost], feed_dict = {X: batch_xs, Y: batch_ys, keep_prob: 0.7})
        
        total_cost += cost_val
        
    print('Epoch: ', '%04d' % (epoch+1), 'Avg. cost = ', '{:.3f}'.format(total_cost/total_batch))
    
print('최적화 완료!')

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: ', sess.run(accuracy, feed_dict = {X: mnist.test.images.reshape(-1,28,28,1), Y: mnist.test.labels, keep_prob : 1}))